In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import neural_tape_controller
import optimize
import pickle
from PIL import Image, ImageDraw,ImageFont
from matplotlib import pyplot as plt
#Положительные числа - положительные награды.
root = '../'
root_dir = root

In [2]:
class econ_game():
  def __init__(self):
    self.table_goods = {}
    good = {'name':'wood','price':np.random.rand()*5+2}
    self.table_goods[good['name']]=good
    good = {'name':'iron','price':np.random.rand()*4+5}
    self.table_goods[good['name']]=good
    good = {'name':'oil','price':np.random.rand()*5+1}
    self.table_goods[good['name']]=good
    good = {'name':'machine tools','price':np.random.rand()*10+50}
    self.table_goods[good['name']]=good
    good = {'name':'furniture','price':np.random.rand()*5+12}
    self.table_goods[good['name']]=good
    good = {'name':'bikes','price':np.random.rand()*9+13}
    self.table_goods[good['name']]=good
    good = {'name':'weapons','price':np.random.rand()*10+16}
    self.table_goods[good['name']]=good
    good = {'name':'cars','price':np.random.rand()*25+40}
    self.table_goods[good['name']]=good
    
    self.table_buildings = {}
    building = {'name':'heavy','price':100,'machine tools':4}
    self.table_buildings[building['name']]=building
    building = {'name':'furniture','price':40,'machine tools':1}
    self.table_buildings[building['name']]=building
    building = {'name':'bikes','price':55,'machine tools':2}
    self.table_buildings[building['name']]=building
    building = {'name':'weapons','price':60,'machine tools':3}
    self.table_buildings[building['name']]=building
    building = {'name':'cars','price':70,'machine tools':4}
    self.table_buildings[building['name']]=building
    
    self.buildings_cnt = {'heavy':0,'furniture':0,'bikes':0,'weapons':0,'cars':0}
    self.store = {}
    for key in self.table_goods.keys():
        self.store[key] = 2
    #self.store['machine tools'] = 2
    #self.store['oil'] = 3
    #self.store['wood'] = 3
    
    #нет добычи, есть лишь производство и торговля
    self.money = 100
    
    self.comission = 0.2
    self.day = 0
    
    self.event = {'forest_crisis':0,'war':0,'eco_activism':0,'industrial_boom':0,'car_boom':0}
    
    
  def step_env(self,draw=False):
    if draw:
        im = Image.new('RGB', (340, 250), (256, 256, 256))
        dr = ImageDraw.Draw(im)
        #Деньги
        dr.text((1, 1),f"${int(self.money)}",'blue')
        #Комиссия
        dr.text((200, 1),f"Day: {np.round(self.day,-1)}",'black')
        #дата
        dr.text((80, 1),f"Comission: {np.round(self.comission*100,1)}%",'blue')
        #Предприятия
        str_factories = ''
        for key in self.buildings_cnt.keys():
            if self.buildings_cnt[key]>0:
                str_factories += f'\n{key}:{self.buildings_cnt[key]}'
        dr.text((230, 20),f"Factories:{str_factories}",'black')
        #Товары
        str_goods = ''
        for key in self.store.keys():
            if self.store[key]>0:
                str_goods += f'\n{key}:{int(self.store[key])}'
        dr.text((1, 20),f"Goods:{str_goods}",'black')
        #Цены
        str_prices = ''
        for key in self.store.keys():
            str_prices += f"\n{key}:{np.round(self.table_goods[key]['price'],1)}"
        dr.text((120, 20),f"Prices:{str_prices}",'black')
        #События
        str_events = ''
        for key in self.event.keys():
            if self.event[key]>0:
                str_events += f'  {key}'
        dr.text((1, 180),f"Events:{str_events}",'red')
        globals()['video'].append(im)
        
        
    #исправление возможных ошибок:
    for key in self.table_goods.keys():
        if self.table_goods[key]['price']<0.5:
            self.table_goods[key]['price'] = 0.5
    #работа производств
    for key in self.buildings_cnt:
        how_many_production = self.buildings_cnt[key]
        if how_many_production>self.store['oil']:
            how_many_production=int(self.store['oil'])
        if key=='heavy':
            if how_many_production>self.store['iron']:
                how_many_production=int(self.store['iron'])
            self.store['iron']-=how_many_production
            self.store['machine tools']+=how_many_production*0.5
        elif key=='furniture':
            if how_many_production>self.store['wood']:
                how_many_production=int(self.store['wood'])
            self.store['wood']-=how_many_production
            self.store['furniture']+=how_many_production
        elif key=='bikes':
            if how_many_production>self.store['iron']:
                how_many_production=int(self.store['iron'])
            self.store['iron']-=how_many_production
            self.store['bikes']+=how_many_production 
        elif key=='weapons':
            if how_many_production>self.store['iron']:
                how_many_production=int(self.store['iron'])
            self.store['iron']-=how_many_production
            self.store['weapons']+=how_many_production 
        elif key=='cars':
            if how_many_production>self.store['iron']:
                how_many_production=int(self.store['iron'])
            self.store['iron']-=how_many_production
            self.store['cars']+=how_many_production*0.5
        self.store['oil']-=how_many_production
    #случайные события
    if self.event['forest_crisis']:
        self.table_goods['wood']['price']*=1.002
    if self.event['war']:
        self.table_goods['weapons']['price']*=1.003
    if self.event['industrial_boom']:
        if np.random.rand()<0.005:
            self.event['war'] = 1
            self.table_goods['weapons']['price']*=1.3
            self.comission *= 1.3
    if self.event['car_boom']:
        if np.random.rand()<0.3:
            self.table_goods['oil']['price']*=1.01
        if np.random.rand()<0.05:
            self.event['car_boom'] = 0
    
    if draw:
        print('$',self.money,self.store,'buildings',self.buildings_cnt,'comission',self.comission,'events',self.event)
  def action(self,action_vector,draw=False):
    self.day += 10
    #if self.buildings_cnt['furniture']<1:
    #    action_vector = [0,1,0, 0,0,0,  2, 0, 5, 1,-10,0]
    #else:
    #    action_vector = [0,0,0, 0,0,0,  3, 0, 3, 0,-10,0]
    #какие вообще варианты, что делать? Здесь будет команда, что строить, что покупать/продавать
    argm = np.argmax(action_vector[:5])#что строить
    buildings_d = ['heavy','furniture','bikes','weapons','cars']
    if action_vector[argm]>-0.001:#строить ли 
      if (self.money>=self.table_buildings[buildings_d[argm]]['price']) and (self.store['machine tools']>=self.table_buildings[buildings_d[argm]]['machine tools']):
        self.money-=self.table_buildings[buildings_d[argm]]['price']
        self.store['machine tools']-=self.table_buildings[buildings_d[argm]]['machine tools']
        
        self.buildings_cnt[buildings_d[argm]]+=1

    argm = np.argmax(action_vector[5:10])#что сносить
    if action_vector[argm]>0:
        if self.buildings_cnt[buildings_d[argm]]>0:
            self.money+=self.table_buildings[buildings_d[argm]]['price']*(1-self.comission)
            self.store['machine tools']+=self.table_buildings[buildings_d[argm]]['machine tools']
            self.buildings_cnt[buildings_d[argm]]-=1
    #продаём/покупаем
    trade_vector = np.array(action_vector[10:10+8])*100
    i = 0
    for k in ['wood','iron','oil','machine tools','furniture','bikes','weapons','cars']:
        trade_vector[i] = np.round(trade_vector[i])
        if trade_vector[i]>0:
            #покупаем
            #if trade_vector[i]>0:
            #    print('try buy',self.table_goods[k]['name'],trade_vector[i],'can',int(self.money/self.table_goods[k]['price']))
            if self.money<self.table_goods[k]['price']*trade_vector[i]:
                trade_vector[i] = int(self.money/self.table_goods[k]['price'])
            self.money-=self.table_goods[k]['price']*trade_vector[i]
            self.store[k] += trade_vector[i]
            #случайные события
            if i==0:
                if (np.random.rand()<trade_vector[0]*0.005) and (self.event['forest_crisis']==0):
                    self.event['forest_crisis'] = 1
                    self.table_goods['wood']['price']*=1.2
                    if draw:
                        print('EVENT forest_crisis! Because trade_vector[0]*0.005 = ',trade_vector[0]*0.005)
            
        elif trade_vector[i]<0:
            #продаём
            trade_vector[i] *= -1
            if self.store[k]<trade_vector[i]:
                trade_vector[i]=self.store[k]
            revenue = self.table_goods[k]['price']*trade_vector[i]*(1-self.comission)
            self.money+=revenue
            self.store[k] -= trade_vector[i]
            #случайные события
            if i==6:
                if (np.random.rand()<trade_vector[6]*0.002) and (self.event['war']==0):
                    self.event['war'] = 1
                    self.table_goods['weapons']['price']*=1.3
                    self.comission *= 1.4
                    if draw:
                        print('EVENT WAR! Because trade_vector[6]*0.005 = ',trade_vector[6]*0.005)
            if i==5:
                if (np.random.rand()<trade_vector[5]*0.002) and (self.event['eco_activism']==0):
                    #много великов
                    self.event['eco_activism'] = 1
                    self.table_goods['oil']['price']*=1.5
                    self.table_goods['wood']['price']*=1.25
                    self.comission *= 0.8
                    if draw:
                        print('EVENT eco_activism! Because trade_vector[5]*0.005 = ',trade_vector[5]*0.005)
            if i==3:
                if (np.random.rand()<trade_vector[3]*0.002) and (self.event['industrial_boom']==0):
                    self.event['industrial_boom'] = 1
                    self.table_goods['oil']['price']*=0.7
                    self.table_goods['wood']['price']*=0.7
                    self.table_goods['iron']['price']*=0.7
                    self.table_goods['machine tools']['price']*=0.7
                    if draw:
                        print('EVENT industrial_boom! Because trade_vector[3]*0.005 = ',trade_vector[3]*0.005)
            if i==7:
                if (np.random.rand()<trade_vector[7]*0.05) and (self.event['car_boom']==0):
                    #много машин продаём
                    self.event['car_boom'] += 1
                    self.table_goods['oil']['price']*=0.9
                    self.table_goods['wood']['price']*=0.8
                    self.table_goods['iron']['price']*=0.8
                    self.table_goods['cars']['price']*=0.9
                    self.comission *= 0.9
                    if draw:
                        print('EVENT industrial_boom! Because trade_vector[7]*0.1 = ',trade_vector[7]*0.1)
        i+=1

In [3]:
def single_game(player,seed=0,rounds=1000,draw=False):
  np.random.seed(seed)
  game = econ_game()
  for t in range(rounds):
    game.step_env(draw=draw)
    if t%10==0:
      #ходит игрок
      action_vector = player.act(game)
      game.action(action_vector,draw=draw)
  scores = game.money
  return scores

In [4]:
state_dim = 22
n_actions = 18

In [5]:
class player_ntm():
  def __init__(self,genom):
    self.controller = neural_tape_controller.nt_controller(tacts=2,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  def act(self,game):

    state = [game.money] + list(game.store.values()) + list(game.buildings_cnt.values())
    for key in game.table_goods.keys():
        state.append(game.table_goods[key]['price'])
    state = np.array(state)

    if np.shape(state)[0]>1:
      shp = np.shape(state)
      state = np.reshape(state,[1,shp[0]])
    action_vector = self.controller.act(state,0,0)
    return action_vector

In [6]:
def check_genom(genom,draw=False,result_write=False):
    q_arr = []
    for i in range(8):
        player = player_ntm(genom)
        score = single_game(player,seed=i,rounds=500,draw=draw)
        q_arr.append(score)
    if draw or result_write:
        print('q_arr',q_arr)
    return (np.mean(q_arr) + np.min(q_arr)+ np.median(q_arr))*0.33 - np.sum(genom**2)*0.000001#взвешенное среднее, с повышенным весом за провалы

In [7]:
nt = neural_tape_controller.nt_controller(input_size=state_dim,output_size=n_actions)
genom = nt.nn.disassemble_genom()

In [8]:
%%time
q = check_genom(genom,False,True)
print(q)

q_arr [0.4803273607473777, 1.7301700398480087]
0.8879690735868623
Wall time: 243 ms


In [ ]:
print(1)
opt = optimize.optimizer(check_genom, genom_size=len(genom),parallel_cores=1)
try:
    with open(root_dir+f'genoms/best_genom_econ.pkl', 'rb') as f:
        genoms = pickle.load(f)
        genom = genoms[-1]
        opt.best_genoms = genoms
    print('loaded successfully')
except Exception:
    nt = neural_tape_controller.nt_controller(input_size=state_dim,output_size=n_actions)
    genom = nt.nn.disassemble_genom()

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open(root+f'genoms/best_genom_econ.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

1
loaded successfully
opt# 0
scores for optimizers augmented [1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10]
chosen evol_wide previous_result: nan per tacts: nan


C:\Users\kilorad\PYTHON\neuroevolution_toolbox_\testing_envs\..\optimize.py:36: RuntimeWarning: Mean of empty slice
  mx.append(np.nanmean(self.history_gain[opt_name])-time_penalty*np.nanmean(self.history_time[opt_name]))
C:\Users\kilorad\Anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


iteration 0 y= [529.49778475 516.91400999 516.91400956 516.91400924]
iteration 1 y= [529.49778475 516.91400999 516.91400956 516.91400924]


In [ ]:
with open(root_dir+f'genoms/best_genom_econ.pkl', 'rb') as f:
    genoms = pickle.load(f)
    genom = genoms[-1]
    opt.best_genoms = genoms
q = check_genom(genom,False,True)
q

In [ ]:
globals()['video'] = []
with open(root_dir+f'genoms/best_genom_econ.pkl', 'rb') as f:
    genoms = pickle.load(f)
    genom = genoms[-1]
    opt.best_genoms = genoms
q = check_genom(genom,True,True)

In [ ]:
#draw video
import cv2
writer = cv2.VideoWriter("./outputvideo.mp4",cv2.VideoWriter_fourcc(*"MP4V"),20,(512,512))
for i in range(0,len(globals()['video']),1):
    resize = cv2.resize(np.array(globals()['video'][i]), (512, 512))
    frame = resize*0
    frame[:,:,0],frame[:,:,1],frame[:,:,2] = resize[:,:,2],resize[:,:,1], resize[:,:,0]
    writer.write(frame)
writer.release()

In [ ]:
#save gif
globals()['video'][0].save(
    root_dir+'./out_videos/economic.gif',
    save_all=True,
    append_images=globals()['video'][1:], 
    optimize=True,
    duration=100,
    loop=0
)